In [2]:
import numpy as np

In [3]:
def compare(fitness_a, fitness_b):
    # if a dominates b, return will be 1, -1 if the contrary
    # if there is no dominance, 0 will be returned
    try:
        m = (fitness_a[0] - fitness_b[0]) / (fitness_a[1] - fitness_b[1])
    except ZeroDivisionError:
        return 0
    if m > 0:
        if fitness_a[0] < fitness_b[0]:
            return 1
        return -1
    return 0
